<a href="https://colab.research.google.com/github/erikaminarikova/Ordinal_Sums/blob/main/Example_6_generation_of_ls_for_california_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import time

Preparing the data

In [11]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import MinMaxScaler

california_housing = fetch_california_housing(as_frame=True)
df = california_housing.frame

columns = df.columns
# normalize the data
scaler = MinMaxScaler()
model = scaler.fit_transform(df)
df_scaled = pd.DataFrame(model, columns = columns)

Generating the linguistic summaries

In [12]:
start_time = time.time()
# testing data
# df = pd.read_csv('data/data_mixed.csv', sep=';')
# running generetion of LS: Q R have S (e.g. (most of/few/about half)old hauses has low price)
def linguistic_summaries(df):
    data = membership_dgree_columns(df)
    x = 0 # which column as R (age of building)
    y = 8 # which column as S (price)
    compute_linguistic_summaries(df, data, x, y)
    return data
result = linguistic_summaries(df_scaled)
print('time:', time.time()-start_time)

Sometimes/About half MedInc_low have MedHouseVal_low t= 0.6044325689703459
Rarely/ Few MedInc_low have MedHouseVal_medium t= 0.3214597397120511
Rarely/ Few MedInc_low have MedHouseVal_high t= 0.08209946436550077
Rarely/ Few MedInc_medium have MedHouseVal_low t= 0.0715732649964386
Sometimes/About half MedInc_medium have MedHouseVal_medium t= 0.46661468270222733
Sometimes/About half MedInc_medium have MedHouseVal_high t= 0.510943274563873
Rarely/ Few MedInc_high have MedHouseVal_low t= 0.013167425099501286
Rarely/ Few MedInc_high have MedHouseVal_medium t= 0.018483221530631866
Most of MedInc_high have MedHouseVal_high t= 0.9710625946328149
time: 3.134422779083252


Definitions of functions needed for generation of LS

In [1]:
def quantifier(x,a,b):
    if x>=b:
        membership_degree=1
    elif x>a and x<b:
        membership_degree=(x-a)/(b-a)
    else:
        membership_degree=0
    return membership_degree

In [2]:
def l_fuzzy_set(x,a,b):
    if x<=a:
        membership_degree=1
    elif b>x>a:
        membership_degree=(b-x)/(b-a)
    else:
        membership_degree=0
    return membership_degree


In [3]:
def r_fuzzy_set(x,a,b):
    if x<=a:
        membership_degree=0
    elif b>x>a:
        membership_degree=(x-a)/(b-a)
    else:
        membership_degree=1
    return membership_degree

In [4]:
def triangular_fuzzy_set(x,a,b,m):
    if x==m:
        membership_degree=1
    elif m>x>a:
        membership_degree=(x-a)/(m-a)
    elif b>x>m:
        membership_degree=(b-x)/(b-m)
    else:
        membership_degree=0
    return membership_degree

In [5]:
def trapeziodal_fuzzy_set(x,a,b,m,n):
    if n>=x>=m:
        membership_degree=1
    elif m>x>a:
        membership_degree=(x-a)/(m-a)
    elif b>x>n:
        membership_degree=(b-x)/(b-n)
    else:
        membership_degree=0
    return membership_degree

In [6]:
def calculate_membership_degree(column):
    initial_list = list(column)
    computed_list_low = []
    computed_list_medium = []
    computed_list_high = []
    for i in initial_list:
        computed_list_low.append(l_fuzzy_set(i, 0.3,0.4))
        computed_list_medium.append(trapeziodal_fuzzy_set(i, 0.3,0.7,0.4,0.6))
        computed_list_high.append(r_fuzzy_set(i, 0.6,0.7))
    final = [computed_list_low, computed_list_medium, computed_list_high]
    return final


In [7]:
#for each column add 3 columns to compute membership to low medium and high (e.g. column price: price_low, price_medium, price_high)
def membership_dgree_columns(df):
    number_of_columns = len(df.columns)
    name = df.columns
    i=0
    while i < number_of_columns:
        final = calculate_membership_degree(df.iloc[:,i])
        final_df = pd.DataFrame(final, index=[name[i] + '_low', name[i] + '_medium', name[i] + '_high'])
        final_df = final_df.transpose()
        df = df.join(final_df)
        i += 1
    return df

In [8]:
# equation for LS calculation
def ls_proporcion(agr_min,sum_a,text1,text2):
    t = sum(agr_min.min(axis=1))/sum(sum_a)
    result_ls= quantifier(t, 0.4, 0.8)
    if result_ls == 0:
        print('Rarely/ Few',text1,"have",text2,"t=",t)
    elif result_ls == 1:
        print('Most of',text1,"have",text2,"t=",t)
    else:
        print('Sometimes/About half',text1,"have",text2,"t=",t)

In [9]:
def compute_linguistic_summaries(df, data, x, y):
    number = len(df.columns)
    x+=1
    y+=1
    first = data.columns[number+(x-1)*3:number+x*3]
    second = data.columns[number+(y-1)*3:number+y*3]
    for i in first:
        for j in second:
            ls_proporcion(data[[i,j]],data[i],i,j)